In [ ]:
import sys
sys.path.append('C:\\Users\\caudillo\\examples')

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix
from IPython.display import clear_output
import matplotlib.pyplot as plt

# Cargando dataset oxford_iiit_pet
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)


In [ ]:
# Normalizar colores
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask

def load_image(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(
        datapoint['segmentation_mask'],
        (128, 128),
        method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
    )
    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask

# The dataset already contains the required training and test splits, so continue to use the same splits
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 128
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

train_images = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

# Create an instance of the Augment class
augment_layer = Augment()

# Build the input pipeline, applying the augmentation after batching the inputs
train_batches = (
    train_images
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .map(lambda x, y: augment_layer(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

test_batches = test_images.batch(BATCH_SIZE)


In [ ]:
# Visualize an image example and its corresponding mask from the dataset
def display(display_list):
    plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

# Muestra un ejemplo de imagen y máscara del conjunto de datos de entrenamiento
for images, masks in train_batches.take(2):  # Toma los primeros 2 lotes de imágenes y máscaras
    sample_image, sample_mask = images[0], masks[0]  # Selecciona la primera imagen y máscara del primer lote
display([sample_image, sample_mask])  # Muestra la imagen y la máscara en una figura

In [ ]:
# Define the GRU model
def gru_model(output_channels: int):
    inputs = tf.keras.layers.Input(shape=[128, 128, 3])

    # Replace the U-Net encoder with GRU layers
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))(inputs)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.GRU(64, return_sequences=True))(x)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))(x)
    x = tf.keras.layers.TimeDistributed(tf.keras.layers.GRU(128, return_sequences=True))(x)

    # Decoder
    up_stack = [
        pix2pix.upsample(64, 3),  # 8x8 -> 16x16
        pix2pix.upsample(32, 3),  # 16x16 -> 32x32
        pix2pix.upsample(16, 3),  # 32x32 -> 64x64
    ]

    skips = x
    for up in up_stack:
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skips])
        skips = x

    # Final layer of the model (decoder)
    last = tf.keras.layers.Conv2DTranspose(
        filters=output_channels, kernel_size=3, strides=2,
        padding='same')  # 64x64 -> 128x128

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

# Train the model
OUTPUT_CLASSES = 3

model = gru_model(output_channels=OUTPUT_CLASSES)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()



In [ ]:
# Callback for visualization
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print('\nSample Prediction after epoch {}\n'.format(epoch+1))

EPOCHS = 10
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples // BATCH_SIZE // VAL_SUBSPLITS

model_history = model.fit(train_batches, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_batches,
                          callbacks=[DisplayCallback()])



In [ ]:
# Plot the training history
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.figure()
plt.plot(model_history.epoch, loss, 'r', label='Training loss')
plt.plot(model_history.epoch, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()





In [ ]:
# Make predictions
def create_mask(pred_mask):
    pred_mask = tf.math.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
                 create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions(test_batches, 3)

In [ ]:
#Parte 2
#Opcional: clases desequilibradas y pesos de clase

try:
  model_history = model.fit(train_batches, epochs=EPOCHS,
                            steps_per_epoch=STEPS_PER_EPOCH,
                            class_weight = {0:2.0, 1:2.0, 2:1.0})
  assert False
except Exception as e:
  print(f"Expected {type(e).__name__}: {e}")


In [ ]:
#model fit


label = tf.constant([0, 0])  # Convertir la lista en un tensor constante
prediction = tf.constant([[-3., 0], [-3, 0]])  # Convertir la lista en un tensor constante
sample_weight = [1, 10]

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
loss_value = loss(label, prediction, sample_weight).numpy()

print(loss_value)

In [ ]:
#Los elementos del conjunto de datos resultantes contienen 3 imágenes cada uno:

train_batches.map(add_sample_weights).element_spec

In [ ]:

#Ahora puede entrenar un modelo en este conjunto de datos ponderado:

weighted_model = unet_model(OUTPUT_CLASSES)
weighted_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

weighted_model.fit(
    train_batches.map(add_sample_weights),
    epochs=1,
    steps_per_epoch=10)